In [84]:
import numpy as np
from model_vae import CVAE, cvae_loss

In [85]:
x_data = np.load('./data/metal.npy')[:,1:]
c_data = np.load('./data/reaction.npy')

In [86]:
from sklearn.model_selection import train_test_split
x_train,x_test,c_train,c_test = train_test_split(x_data,c_data,test_size = 0.4,random_state = 21)
x_val,x_test,c_val,c_test = train_test_split(x_test,c_test,test_size = 0.5,random_state=21)

In [87]:
from sklearn.preprocessing import MinMaxScaler
x_scaler = MinMaxScaler()
c_scaler = MinMaxScaler()
x_train = x_scaler.fit_transform(x_train)
c_train = c_scaler.fit_transform(c_train)
x_val,x_test = [x_scaler.transform(x) for x in [x_val,x_test]]
c_val,c_test = [c_scaler.transform(c) for c in [c_val,c_test]]

In [88]:
x_train,x_val,x_test = [torch.tensor(x,dtype = torch.float32) for x in [x_train,x_val,x_test]]
c_train,c_val,c_test = [torch.tensor(x,dtype = torch.float32) for x in [c_train,c_val,c_test]]

In [89]:
from torch.utils.data import DataLoader, TensorDataset
train_data = [x_train,c_train]
val_data =  [x_val,c_val]
test_data =  [x_test,c_test]
## 텐서로 바꾸기
train_data = TensorDataset(*train_data)
val_data = TensorDataset(*val_data)
test_data = TensorDataset(*test_data)

In [90]:
data = [train_data,val_data,test_data]
batch_size = 64
train_loader,val_loader,test_loader = [DataLoader(x,batch_size=batch_size,shuffle = False) for x in data]

In [91]:
x_dim = x_data.shape[1]
c_dim = c_data.shape[1]
print(x_dim,c_dim)

23 9


In [96]:
type(train_loader)

torch.utils.data.dataloader.DataLoader

In [93]:
model = CVAE(x_dim,c_dim,z_dim = 16)
# x_train.shape,c_train.shape
x_hat, mu,logvar = model(x_train,c_train)

RuntimeError: mat1 and mat2 shapes cannot be multiplied (3544x32 and 64x23)

In [ ]:
learning_rate=  1e-3
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = CVAE(x_dim,c_dim).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)


In [ ]:
epochs = 200
train_losses = []
val_losses =[]
for epoch in range(1,epochs+1):
    model.train()
    total_train_loss = 0.0
    for batch_x,batch_c in train_loader:
        batch_x = batch_x.to(device)
        batch_c = batch_c.to(device)
        x_hat,mu,logvar = model(batch_x,batch_c)
        loss,recon,kl = cvae_loss(x_hat,batch_x,mu,logvar,beta = 0.01)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        total_train_loss +=loss.item()
    avg_train_loss = total_train_loss/len(train_loader)
    # val
    model.eval()
    total_val_loss = 0.0
    with torch.no_grad():
        for batch_val_x,batch_val_c in val_loader:
            batch_val_x = batch_val_x.to(device)
            batch_val_c = batch_val_c.to(device)
            v_x_hat,v_mu,v_logvar = model(batch_val_x,batch_val_c)
            v_loss,_,_ = cvae_loss(v_x_hat,batch_val_x)
            total_val_loss +=loss.item()
    avg_val_loss = total_val_loss/len(val_loader)
    train_losses.append(avg_train_loss)
    val_losses.append(avg_val_loss)
    if epoch %20 ==0 or epoch ==1:
        print(f'[{epoch}/{epochs}] train_loss:{avg_train_loss:.4f}, val_loss:{avg_val_loss:.4f}')
        